In [ ]:
from math import isqrt
import os.path
from time import perf_counter

import numpy as np
from scipy.special import gamma, gammaln, gammainc
import scipy.linalg as la
import scipy.integrate as spi
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4
import matplotlib.animation as animation

import bin_model as bm

In [ ]:
# Define physical constants.
RHO_WATER = 1000. # Density of water (kg/m^3)
RHO_AIR = 1.2 # Density of air (kg/m^3)

# Model physical parameters.
RAIN_D = 8.e-5 # Cutoff diameter between particle sizes defined as cloud vs. rain (m).

# Grid parameters
D_MIN = 1.e-6 # Minimum particle diameter (m).
D_MAX = 1.6384e-2 # Maximum particle diameter (m).
NUM_BINS = 126 # Number of evenly spaced bins in the simulation

# Initial conditions
INITIAL_MASS = 1.e-3 # Initial mass concentration (kg/m^3)
INITIAL_NC = 100. # Initial number concentration (cm^-3)
INITIAL_NU = 6. # Shape parameter for initial condition

# Numerical tuning parameters.
STD_DIAMETER = 1.e-4 # Internal scaling for particle size (m)
MASS_CONC_SCALE = 1.e-3
# Long's kernel magnitude kc (m^3/kg^2/s)
long_kernel_size = 9.44e9
TIME_SCALE = 1. / (long_kernel_size * ((np.pi*RHO_WATER/6.)*STD_DIAMETER**3)
                       * MASS_CONC_SCALE)

In [ ]:
const = bm.ModelConstants(rho_water=RHO_WATER, rho_air=RHO_AIR, std_diameter=STD_DIAMETER,
                          rain_d=RAIN_D, mass_conc_scale=MASS_CONC_SCALE,
                          time_scale=TIME_SCALE)
#kernel = bm.LongKernel(const)
kernel = bm.HallKernel(const, 'ScottChen')
grid = bm.GeometricMassGrid(const, d_min=D_MIN, d_max=D_MAX, num_bins=NUM_BINS)

In [ ]:
ktens = bm.KernelTensor(kernel, grid)

In [ ]:
mass_init = INITIAL_MASS
m3_scaled_init = mass_init / const.std_mass # kg^-1
m3_init = mass_init / (const.rho_water * np.pi/6.) # m^3 / kg 3rd moment
m0_init = INITIAL_NC * 1.e6 * const.rho_air # kg^-1 number concentration
lambda_init = ( m0_init * gamma(INITIAL_NU + 3)/ (m3_init * gamma(INITIAL_NU)) )**(1./3.) # m^-1 scale parameter

In [ ]:
dsd_deriv_names = ['lambda', 'nu']
dsd_deriv_scales = [const.std_diameter, 1.]
desc = bm.ModelStateDescriptor(const, grid,
                               dsd_deriv_names=dsd_deriv_names,
                               dsd_deriv_scales=dsd_deriv_scales)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((2, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
raw = desc.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state = bm.ModelState(desc, raw)

In [ ]:
nb = grid.num_bins
y_init = np.zeros((3*nb,))
y_init[:nb] = desc.dsd_raw(raw)
y_init[nb:2*nb] = desc.dsd_deriv_raw(raw, 'lambda')
y_init[2*nb:] = desc.dsd_deriv_raw(raw, 'nu')

In [ ]:
tscale = const.time_scale
end_time = 3600.
num_time_steps = 120
dt = end_time / num_time_steps
integrator = bm.RK45Integrator(const, dt)
exp = integrator.integrate(end_time, init_state, [ktens])
t_eval = exp.times
states = exp.states
all_y = np.zeros((len(t_eval), 3*grid.num_bins,))
for i in range(len(t_eval)):
    raw = states[i].raw
    all_y[i,:grid.num_bins] = desc.dsd_raw(raw)
    all_y[i,grid.num_bins:2*grid.num_bins] = desc.dsd_deriv_raw(raw,
                                                                'lambda')
    all_y[i,2*grid.num_bins:] = desc.dsd_deriv_raw(raw, 'nu')

In [ ]:
bin_midpoints = 0.5 * (grid.bin_bounds[:-1] + grid.bin_bounds[1:])
plot_lxs = 2. + bin_midpoints / (3.*np.log(10))

In [ ]:
mass_convert = mass_init * 1.e3 * (3.*np.log(10))
ymin = 0.
ymax = 0.4*mass_convert
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + grid.lx_min/(3.*np.log(10)), 2. + grid.lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.set_ylabel("$dm/dlog_{10}(D)$ (g/kg)")
ax.grid()

line, = ax.plot(plot_lxs, y_init[:grid.num_bins], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(np.log10(const.rain_d)+6., ymin, ymax, 'k')

def animate(i):
    thisy = mass_convert*all_y[i,:grid.num_bins]
    line.set_data(plot_lxs, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

ani = animation.FuncAnimation(
    fig, animate, len(t_eval), interval=dt*1000*(5./end_time), blit=True)
ani.save("mass_evolution.gif")
plt.show()

In [ ]:
m3_init

In [ ]:
print(init_state.dsd_moment(3))

In [ ]:
final = states[-1].dsd_moment(3)
fallout = states[-1].fallout()/(np.pi*const.rho_water/6.)
print(final+fallout, final, fallout)

In [ ]:
autos = np.zeros((num_time_steps+1,))
accrs = np.zeros((num_time_steps+1,))
cloud_lx = np.log(const.rain_m)
cloud_idx = grid.find_bin(cloud_lx)
if (grid.bin_bounds[cloud_idx+1] - cloud_lx) < 1.e-10:
    cloud_idx += 1
cloud_vector = np.zeros((nb,))
cloud_vector[:cloud_idx] = 1.
for i in range(num_time_steps+1):
    autos[i], accrs[i] = states[i].rain_prod_breakdown(ktens, cloud_vector)

In [ ]:
wv0 = grid.moment_weight_vector(0)
wv3 = grid.moment_weight_vector(3)
wv6 = grid.moment_weight_vector(6)
wv9 = grid.moment_weight_vector(9)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wv0, bm.LogTransform(), db_scale),
    (wv3, bm.LogTransform(), db_scale),
    (wv6, bm.LogTransform(), db_scale),
    (wv9, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one hour
perturbation_rate = error_rate**2 * np.eye(4) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
desc2 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw2 = desc2.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state2 = bm.ModelState(desc2, raw2)

In [ ]:
exp2 = integrator.integrate(end_time, init_state2, [ktens])
states2 = exp2.states

In [ ]:
moments = [0, 3, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states2[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
wvc0 = grid.moment_weight_vector(0, cloud_only=True)
wvc3 = grid.moment_weight_vector(3, cloud_only=True)
wvr0 = grid.moment_weight_vector(0, rain_only=True)
wvr3 = grid.moment_weight_vector(3, rain_only=True)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wvc0, bm.LogTransform(), db_scale),
    (wvc3, bm.LogTransform(), db_scale),
    (wvr0, bm.LogTransform(), db_scale),
    (wvr3, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one day
perturbation_rate = error_rate**2 * np.eye(4) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
desc3 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw3 = desc3.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state3 = bm.ModelState(desc3, raw3)

In [ ]:
exp3 = integrator.integrate(end_time, init_state3, [ktens])
states3 = exp3.states

In [ ]:
nmom = 4
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states3[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Rain M0", "Rain M3"]
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n])
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
wvc0 = grid.moment_weight_vector(0, cloud_only=True)
wvc3 = grid.moment_weight_vector(3, cloud_only=True)
wvc6 = grid.moment_weight_vector(6, cloud_only=True)
wvr0 = grid.moment_weight_vector(0, rain_only=True)
wvr3 = grid.moment_weight_vector(3, rain_only=True)
wvr6 = grid.moment_weight_vector(6, rain_only=True)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wvc0, bm.LogTransform(), db_scale),
    (wvc3, bm.LogTransform(), db_scale),
    (wvc6, bm.LogTransform(), db_scale),
    (wvr0, bm.LogTransform(), db_scale),
    (wvr3, bm.LogTransform(), db_scale),
    (wvr6, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one day
perturbation_rate = error_rate**2 * np.eye(6) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
correction_time = 5.
desc4 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate,
                                correction_time=correction_time)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw4 = desc4.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state4 = bm.ModelState(desc4, raw4)

In [ ]:
exp4 = integrator.integrate(end_time, init_state4, [ktens])
states4 = exp4.states

In [ ]:
nmom = 6
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states4[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Cloud M6", "Rain M0", "Rain M3", "Rain M6"]
colors = ['tab:blue', 'tab:orange', 'tab:purple', 'tab:green', 'tab:red', 'tab:brown']
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n], color=colors[n])
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
wv0 = grid.moment_weight_vector(0)
wv3 = grid.moment_weight_vector(3)
wv6 = grid.moment_weight_vector(6)
wv9 = grid.moment_weight_vector(9)
wv12 = grid.moment_weight_vector(12)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wv0, bm.LogTransform(), db_scale),
    (wv3, bm.LogTransform(), db_scale),
    (wv6, bm.LogTransform(), db_scale),
    (wv9, bm.LogTransform(), db_scale),
    (wv12, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one hour
perturbation_rate = error_rate**2 * np.eye(5) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
correction_time = 5.
desc5 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate,
                                correction_time=correction_time)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw5 = desc5.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state5 = bm.ModelState(desc5, raw5)

In [ ]:
exp5 = integrator.integrate(end_time, init_state5, [ktens])
states5 = exp5.states

In [ ]:
moments = [0, 3, 6, 9, 12]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states5[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
covar_volume_single_cat = np.zeros((num_time_steps+1,))
covar_volume_two_cat_2m = np.zeros((num_time_steps+1,))
covar_volume_two_cat_3m = np.zeros((num_time_steps+1,))
covar_volume_one_cat_5m = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    perturb_cov = states2[i].perturb_cov()
    covar_volume_single_cat[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = states3[i].perturb_cov()
    covar_volume_two_cat_2m[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = states4[i].perturb_cov()
    covar_volume_two_cat_3m[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = states5[i].perturb_cov()
    covar_volume_one_cat_5m[i] = np.prod(la.svdvals(perturb_cov)[:4])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, covar_volume_single_cat, label='1-cat 0-3-6-9')
plt.plot(t_eval, covar_volume_one_cat_5m, label='1-cat 0-3-6-9-12')
plt.plot(t_eval, covar_volume_two_cat_2m, label='2-cat c03r03')
plt.plot(t_eval, covar_volume_two_cat_3m, label='2-cat c036r036')
#ymax = 4.5e-6
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
ddsddt_single_cat = exp2.ddsddt
ddsddt_two_cat_2m = exp3.ddsddt
ddsddt_two_cat_3m = exp4.ddsddt
ddsddt_one_cat_5m = exp5.ddsddt
zeta_covar_single_cat = exp2.zeta_cov
zeta_covar_two_cat_2m = exp3.zeta_cov
zeta_covar_two_cat_3m = exp4.zeta_cov
zeta_covar_one_cat_5m = exp5.zeta_cov

In [ ]:
cm3_amount = np.zeros((num_time_steps+1,))
single_cat_cm3_uncertainty = np.zeros((num_time_steps+1,))
two_cat_2m_cm3_uncertainty = np.zeros((num_time_steps+1,))
two_cat_3m_cm3_uncertainty = np.zeros((num_time_steps+1,))
one_cat_5m_cm3_uncertainty = np.zeros((num_time_steps+1,))
cm3_vector = grid.moment_weight_vector(3, cloud_only=True)
for i in range(num_time_steps+1):
    cm3_amount[i], deriv = states2[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_single_cat[i,:])
    single_cat_cm3_uncertainty[i] = deriv @ zeta_covar_single_cat[i,:,:] @ deriv
    _, deriv = states3[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_2m[i,:])
    two_cat_2m_cm3_uncertainty[i] = deriv @ zeta_covar_two_cat_2m[i,:,:] @ deriv
    _, deriv = states4[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_3m[i,:])
    two_cat_3m_cm3_uncertainty[i] = deriv @ zeta_covar_two_cat_3m[i,:,:] @ deriv
    _, deriv = states4[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_one_cat_5m[i,:])
    one_cat_5m_cm3_uncertainty[i] = deriv @ zeta_covar_one_cat_5m[i,:,:] @ deriv
scale = const.mass_conc_scale / mass_init
single_cat_cm3_uncertainty *= scale
two_cat_2m_cm3_uncertainty *= scale
two_cat_3m_cm3_uncertainty *= scale
one_cat_5m_cm3_uncertainty *= scale
cm3_amount *= scale

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, cm3_amount, 'k', label='Total remaining')
plt.plot(t_eval, single_cat_cm3_uncertainty, label='1-cat 0-3-6-9')
plt.plot(t_eval, one_cat_5m_cm3_uncertainty, label='1-cat 0-3-6-9-12')
plt.plot(t_eval, two_cat_2m_cm3_uncertainty, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_cm3_uncertainty, label='2-cat c036r036')
#ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
single_cat_m6 = np.zeros((num_time_steps+1,))
single_cat_m6_uncertainty = np.zeros((num_time_steps+1,))
two_cat_2m_m6 = np.zeros((num_time_steps+1,))
two_cat_2m_m6_uncertainty = np.zeros((num_time_steps+1,))
two_cat_3m_m6 = np.zeros((num_time_steps+1,))
two_cat_3m_m6_uncertainty = np.zeros((num_time_steps+1,))
one_cat_5m_m6 = np.zeros((num_time_steps+1,))
one_cat_5m_m6_uncertainty = np.zeros((num_time_steps+1,))
m6_vector = grid.moment_weight_vector(6)
for i in range(num_time_steps+1):
    single_cat_m6[i], deriv = states2[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_single_cat[i,:])
    single_cat_m6_uncertainty[i] = deriv @ zeta_covar_single_cat[i,:,:] @ deriv
    two_cat_2m_m6[i], deriv = states3[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_2m[i,:])
    two_cat_2m_m6_uncertainty[i] = deriv @ zeta_covar_two_cat_2m[i,:,:] @ deriv
    two_cat_3m_m6[i], deriv = states4[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_3m[i,:])
    two_cat_3m_m6_uncertainty[i] = deriv @ zeta_covar_two_cat_3m[i,:,:] @ deriv
    one_cat_5m_m6[i], deriv = states5[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_one_cat_5m[i,:])
    one_cat_5m_m6_uncertainty[i] = deriv @ zeta_covar_one_cat_5m[i,:,:] @ deriv
scale = const.mass_conc_scale * const.std_diameter**6 / const.std_mass
single_cat_m6_uncertainty = np.sqrt(single_cat_m6_uncertainty) * scale
two_cat_2m_m6_uncertainty = np.sqrt(two_cat_2m_m6_uncertainty) * scale
two_cat_3m_m6_uncertainty = np.sqrt(two_cat_3m_m6_uncertainty) * scale
one_cat_5m_m6_uncertainty = np.sqrt(one_cat_5m_m6_uncertainty) * scale
single_cat_m6 *= scale
two_cat_2m_m6 *= scale
two_cat_3m_m6 *= scale
one_cat_5m_m6 *= scale

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m6, label='Total M6', color='k')
plt.plot(t_eval, single_cat_m6_uncertainty, label='1-cat 0-3-6-9')
plt.plot(t_eval, one_cat_5m_m6_uncertainty, label='1-cat 0-3-6-9-12')
plt.plot(t_eval, two_cat_2m_m6_uncertainty, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m6_uncertainty, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m6, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_m6, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m6, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
single_cat_m0 = np.zeros((num_time_steps+1,))
single_cat_m0_uncertainty = np.zeros((num_time_steps+1,))
two_cat_2m_m0 = np.zeros((num_time_steps+1,))
two_cat_2m_m0_uncertainty = np.zeros((num_time_steps+1,))
two_cat_3m_m0 = np.zeros((num_time_steps+1,))
two_cat_3m_m0_uncertainty = np.zeros((num_time_steps+1,))
one_cat_5m_m0 = np.zeros((num_time_steps+1,))
one_cat_5m_m0_uncertainty = np.zeros((num_time_steps+1,))
m0_vector = grid.moment_weight_vector(0)
for i in range(num_time_steps+1):
    single_cat_m0[i], deriv = states2[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_single_cat[i,:])
    single_cat_m0_uncertainty[i] = deriv @ zeta_covar_single_cat[i,:,:] @ deriv
    two_cat_2m_m0[i], deriv = states3[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_2m[i,:])
    two_cat_2m_m0_uncertainty[i] = deriv @ zeta_covar_two_cat_2m[i,:,:] @ deriv
    two_cat_3m_m0[i], deriv = states4[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_3m[i,:])
    two_cat_3m_m0_uncertainty[i] = deriv @ zeta_covar_two_cat_3m[i,:,:] @ deriv
    one_cat_5m_m0[i], deriv = states4[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_one_cat_5m[i,:])
    one_cat_5m_m0_uncertainty[i] = deriv @ zeta_covar_one_cat_5m[i,:,:] @ deriv
scale = const.mass_conc_scale / const.std_mass
single_cat_m0_uncertainty = np.sqrt(single_cat_m0_uncertainty) * scale
two_cat_2m_m0_uncertainty = np.sqrt(two_cat_2m_m0_uncertainty) * scale
two_cat_3m_m0_uncertainty = np.sqrt(two_cat_3m_m0_uncertainty) * scale
one_cat_5m_m0_uncertainty = np.sqrt(one_cat_5m_m0_uncertainty) * scale
single_cat_m0 *= scale
two_cat_2m_m0 *= scale
two_cat_3m_m0 *= scale

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m0, label='Total M0', color='k')
plt.plot(t_eval, single_cat_m0_uncertainty, label='1-cat 0-3-6-9')
plt.plot(t_eval, one_cat_5m_m0_uncertainty, label='1-cat 0-3-6-9-12')
plt.plot(t_eval, two_cat_2m_m0_uncertainty, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m0_uncertainty, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m0, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_m0, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m0, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')